In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from fact.io import read_h5py

exec(open('/home/msackel/Desktop/gammaClassification/programm/theta_cut/theta_cut.py').read())
exec(open('/home/msackel/Desktop/gammaClassification/programm/model_significance/model_significance.py').read())


with open('/home/msackel/Desktop/gammaClassification/config/feature.yaml') as f:
    feature = yaml.load(f)

mess_hadron= read_h5py(
    '/home/msackel/Desktop/gammaClassification/data/raw_data/crab_precuts.hdf5',
    key='events',
    columns=list(feature)+['theta_deg']
    )

mc_hadron= pd.read_hdf('/home/msackel/Desktop/gammaClassification/data/raw_data/proton_precuts.hdf5')[feature+['theta_deg']]

In [16]:
mess_hadron= mess_hadron.query('theta_deg>='+str(np.sqrt(0.9)))[:100000]
mc_hadron= mc_hadron.query('theta_deg>='+str(np.sqrt(0.9)))[:100000]
mess_hadron['label']=1
mc_hadron['label']=0
data = pd.concat([mess_hadron.drop('theta_deg', axis=1), mc_hadron.drop('theta_deg', axis=1)])
X_train, X_test, y_train, y_test = train_test_split(data.drop(['label'], axis=1), data.label, random_state=42)

In [20]:
Tree = RandomForestClassifier(max_depth=15,criterion='entropy', n_estimators=100, n_jobs=25)

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
selector = RFE(Tree, 1, step=1)
selector = selector.fit(X_train, y_train)
selector.support_ 

print('Ranking:', selector.ranking_)
print('Feature:', feature)

Ranking: [ 8  6  3 10  7  9  1 12 11  2  4  5]
Feature: ['conc_core', 'concentration_one_pixel', 'concentration_two_pixel', 'leakage', 'leakage2', 'size', 'width', 'num_islands', 'num_pixel_in_shower', 'ph_charge_shower_max', 'ph_charge_shower_mean', 'ph_charge_shower_variance']
